In [1]:
import torch
from torch import nn, Tensor
from transformer_model import *
from utils import *
import math

from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [2]:
train_iter = WikiText2(split = 'train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials = ['<unk>'])
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    data = [torch.tensor(vocab(tokenizer(item)), dtype = torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))  # numel() 返回数组中元素的个数

In [3]:
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

In [4]:
# generate_square_subsequent_mask(10)

In [5]:
# test_tensor = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])
# test_view = test_tensor.view((2, -1)).t().contiguous()
# print(test_view)

In [6]:
# test_tensor[3] *= 10
# print(test_tensor)
# print(test_view)

In [7]:
batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)
val_data = batchify(val_data, batch_size)
test_data = batchify(test_data, batch_size)

In [8]:
# train_data[1]

In [9]:
# inp, target = get_batch(train_data, 1)

In [10]:
# inp.size(), target.size()

In [11]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability

model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(DEVICE)

In [12]:
# model

In [13]:
import copy
import time

In [14]:
criterion = nn.CrossEntropyLoss()
lr = 5.0
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma = 0.95)

In [15]:
def train(model: nn.Module) -> None:
    model.train()
    total_loss = 0
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(BPTT).to(DEVICE)
    
    num_batches = len(train_data) // BPTT
    for batch, i in enumerate(range(0, train_data.size(0) - 1, BPTT)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != BPTT:
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        
        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

In [16]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()
    total_loss = 0
    src_mask = generate_square_subsequent_mask(BPTT).to(DEVICE)
    with torch.no_grad():
        for i in ragne(0, eval_data.size(0) - 1, BPTT):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != BPTT:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [ ]:
best_val_loss = float('inf')
epochs = 5
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)
    scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 717.59 | loss  8.17 | ppl  3526.55
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 711.54 | loss  6.91 | ppl   997.61
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 708.45 | loss  6.48 | ppl   649.93
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 709.21 | loss  6.31 | ppl   551.47
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 708.91 | loss  6.19 | ppl   488.58
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 708.40 | loss  6.16 | ppl   473.17
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 707.83 | loss  6.12 | ppl   453.25
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 708.26 | loss  6.11 | ppl   449.57
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 5333.09 | loss  6.03 | ppl   415.19
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 709.08 | loss  6.02 | ppl   410.99
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 711.16 | loss  5.90 | ppl   363.89
